In [ ]:
#!jupyter nbconvert --to=python pCO2SOCAT_python3.ipynb

In [1]:
class pCO2comp:
    '''
    class pCO2SOCATcomp(resultpath,savepath,meshpath,ncpath,first_year,last_year,
                mapproj='pc',savefig=False,layerwise=False, runname='fesom')
    '''
    
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 mapproj='pc',
                 SOCATvar='TAlk',
                 cmap='viridis',
                 savefig=False,
                 cmap_extension='both',
                 verbose=False,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.cmap = cmap
        self.SOCATvar = SOCATvar
        self.cmap_extension = cmap_extension
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        self.verbose = verbose

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm
        from load_interp_SOCAT import SOCATdata
        
        # derive SOCAT mapping projection -------------------------------------------------------------------------------------
        if((self.mapproj != 'pc') & (self.mapproj != 'rob')):
            print('Projection for plotting SOCAT data is not supported! Choose "pc" or "rob".\nprojection set to "pc"')
            self.mapproj == 'rob'
        
        box=[-180, 180, -90, 90]
        self.mapproj = pf.get_proj(self.mapproj)
 
        # load FESOM mesh -------------------------------------------------------------------------------------
        years = np.arange(self.fyear, self.lyear+1,1)
        

        # load FESOM data -------------------------------------------------------------------------------------
        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        unitfesom = 'pCO$_2$ [$\mu$atm]' # equals to mumol/L
        
        # interpolate Glodap data -------------------------------------------------------------------------------------
        SOCAT_input = SOCATdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.SOCATvar, 
                                self.fyear, self.lyear, get_overview=False)
        socat_int = SOCAT_input.socat_int
        print(np.shape)
        time_soccat = SOCAT_input.time
            
        int = np.intersect1d(years, np.arange(1970,2020))
        
        if np.size(int) != 0:
            previous_year=-1
            previous_month=-1
            FESOM = np.empty((len(mesh.x2),len(time_soccat)))
            SOCAT = np.empty((len(mesh.x2),len(time_soccat)))
            DIFF = np.empty((len(mesh.x2),len(time_soccat)))
            for i in range(len(time_soccat)):
                current_time = time_soccat[i]
                current_year = current_time.year
                current_month = current_time.month
                
                if current_year != previous_year:
                    pCO2fesom = pf.get_data(resultpath, "pCO2s", current_year, mesh, 
                                   how=None, compute=True, runid=self.runname, silent=True)

                if current_month != previous_month:
                    fesom_monthly = pCO2fesom[current_month-1,:]

                # apply sea mask to Glodap as in FESOM ----------------------------------------------------------------------------------
                # assumption: there is no ocean where value in FESOM == 0
                socat_int_ma = np.copy(socat_int[:,i])
                socat_int_ma[fesom_monthly == 0] = 0

                diff = fesom_monthly - socat_int_ma

                DIFF[:,i]=diff
                SOCAT[:,i]=socat_int_ma
                FESOM[:,i]=fesom_monthly

                previous_year = current_year
                previous_month = current_month

            DIFF_mean = np.nanmean(DIFF, axis= 1)
            SOCAT_mean = np.nanmean(SOCAT, axis= 1)
            FESOM_mean = np.nanmean(FESOM, axis= 1)
        elif np.size(int) == 0:
            SOCAT_mean = SOCAT_input.socat_int_mean
            FESOM_mean = pf.get_data(resultpath, "pCO2s", years, mesh, 
                                   how='mean', compute=True, runid=self.runname, silent=True)
            DIFF_mean = FESOM_mean - SOCAT_mean
        
        # ==============================================================================
        # plot FESOM and SOCAT   
        if(self.verbose):
            print('\nPlotting pCO2\nFESOM min = {0}, max = {1}'.format(
                        np.nanmin(FESOM_mean),np.nanmax(FESOM_mean)))
            print('SOCAT min = {0}, max = {1}'.format(
                    np.nanmin(SOCAT_mean),np.nanmax(SOCAT_mean)))
        
        if ((self.fyear < 1970) and (self.lyear > 2020)):
            labelsocat = 'SOCAT (1970-2017)'
            label_diff = 'FESOM - SOCAT (1970-2020)'
        elif ((self.fyear < 1970) and (self.lyear <= 2020) and (self.lyear >= 1970)):
            labelsocat = 'SOCAT (1970-{0})'.format(last_year)
            label_diff = 'FESOM - SOCAT (1970-{0})'.format(last_year)
        elif ((self.fyear >= 1970) and (self.lyear > 2020)):
            labelsocat = 'SOCAT ({0}-2020)'.format(first_year)
            label_diff = 'FESOM - SOCAT ({0}-2020)'.format(first_year)
        elif ((self.lyear < 1970)):
            labelsocat = 'SOCAT (1970-2020)'
            label_diff = 'FESOM - SOCAT (no overlap)'
        elif ((self.fyear > 2020)):
            labelsocat = 'SOCAT (1970-2020)'
            label_diff = 'FESOM - SOCAT (no overlap)'
        else:
            labelsocat = 'SOCAT ({0}-{1})'.format(self.fyear,self.lyear)
            label_diff = 'FESOM - SOCAT ({0}-{1})'.format(self.fyear,self.lyear)
            
        unitsocat = 'pCO$_2$ [$\mu$atm]' 
            
        if plotting:
            fig = plt.figure(figsize=(15,12), constrained_layout=False)
            axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))
                    
            m1 = axes['A']
            levels = np.arange(140,520,20)
            f1 = pf.subplot(mesh, fig, m1, [FESOM_mean],
                            levels = levels,
                            units=unitsocat, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension='both',
                            titles=labelfesom,
                            box=box,
                           )
                    
            m2 = axes['B']
            f2 = pf.subplot(mesh, fig, m2, [SOCAT_mean], 
                            levels = levels,
                            units=unitsocat, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                           cmap_extension='both',
                            titles=labelsocat,
                            box=box,
                           )
                    
            cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
            cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
            cbar1.set_label(unitfesom, fontsize=18)
            cbar1.ax.tick_params(labelsize=18)
        
            m3 = axes['C']

            levels_diff = np.arange(-120,130,10)
            f3 = pf.subplot(mesh, fig, m3, [DIFF_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitsocat, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles=label_diff,
                            box=box,
                           )
            
            m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
            m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                        size=30, weight='bold')
            m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                        size=30, weight='bold')
            
            fig.subplots_adjust(bottom=0.02)
            cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
            cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
            cbar2.set_label(unitfesom, fontsize=18)
            cbar2.ax.tick_params(labelsize=18)

                
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'pCO2_SOCAT'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'pCO2_SOCAT'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
            plt.show(block=False)
            
            
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(SOCAT_mean)): print('WARNING: The interpolated Glodap field contains NaNs')
                if np.isnan(np.min(FESOM_mean)): print('WARNING: The interpolated FESOM field contains NaNs')

                aux = np.where(np.isfinite(SOCAT_mean))

                title = 'Taylor Diagram for pCO$_2$'
                plt_Taylor_norm(SOCAT_mean[aux],FESOM_mean[aux],mask=True,title=title)

                # fig export  
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'pCO2_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                                dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'pCO2_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                                bbox_inches='tight')
                plt.show(block=False)
                
            if output:
                    self.pco2_fesom = FESOM_mean
                    self.pco2_socat = SOCAT_mean
                    self.pco2_diff  = DIFF_mean